In [1]:
import torch
import torch.nn as nn
import torchvision as tv
import torch.nn.functional as F

import numpy as np
import matplotlib.pyplot as plt
import cv2
import os
from tqdm import tqdm

In [2]:
torch.cuda.is_available()

True

In [3]:
trans = tv.transforms.Compose([
    tv.transforms.ToTensor()
])

In [4]:
ds_mnist = tv.datasets.MNIST("/.datasets", download=True, transform=trans)

In [5]:
dataloader = torch.utils.data.DataLoader(
    ds_mnist, 16, shuffle=True,
    num_workers=0, drop_last=True
)

In [6]:
class NeuralNumbers(nn.Module):
  def __init__(self):
    super().__init__()
    flat = nn.Flatten()
    linear1 = nn.Linear(28*28, 128)
    linear2 = nn.Linear(128, 10)
    activation = nn.ReLU()
    self.model = nn.Sequential(flat, linear1, activation, linear2)

  def forward(self, x):
    return self.model(x)

In [7]:
def count_parameters(model):
  return sum(p.numel() for p in model.parameters() if p.requires_grad)

In [13]:
model = NeuralNumbers()
devise = "cuda" if torch.cuda.is_available() else "cpu"
model = model.to(devise)

In [14]:
count_parameters(model)

101770

In [15]:
loss_fn = nn.CrossEntropyLoss()
loss_fn = loss_fn.to(devise)

In [16]:
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3, momentum=0.9)

In [19]:
def accuracy(labels, prediction):
  score = F.softmax(prediction.detach().cpu()).numpy().argmax(1) == labels.cpu().numpy().argmax(1)
  return score.mean()

In [18]:
epochs = 10
for i in range(epochs):
  loss_val = 0
  accuracy_val = 0
  for img, labels in (pbar := tqdm(dataloader)):
    optimizer.zero_grad()
    
    img = img.to(devise)
    labels = labels.to(devise)
    
    labels = F.one_hot(labels, 10).float()
    outp = model(img)
    loss = loss_fn(outp, labels)
    loss.backward()
    optimizer.step()
    
    with torch.no_grad():
        loss_item = loss.item()
        loss_val += loss_item
        accuracy_curr = accuracy(labels, outp)
        accuracy_val += accuracy_curr

    pbar.set_description(f"loss {loss_item:.4f}, accuracy {accuracy_curr}")
  print(loss_val / len(dataloader))
  print(accuracy_val / len(dataloader))

  0%|                                                                                         | 0/3750 [00:00<?, ?it/s]C:\Users\bratk\AppData\Local\Temp\ipykernel_30224\3647358103.py:2: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  score = (F.softmax(prediction.detach().cpu()).numpy() > 0.5) == (label.cpu().numpy() > 0.5)
loss 0.2422, accuracy 0.9875: 100%|███████████████████████████████████████████████| 3750/3750 [00:26<00:00, 141.30it/s]


0.6305729167699814
0.965256666666672


loss 0.4822, accuracy 0.96875: 100%|██████████████████████████████████████████████| 3750/3750 [00:24<00:00, 153.48it/s]


0.3072529208958149
0.9831600000000094


loss 0.2420, accuracy 0.98125: 100%|██████████████████████████████████████████████| 3750/3750 [00:25<00:00, 149.98it/s]


...

0.14532507771415015
0.9920600000000164


loss 0.0666, accuracy 1.0: 100%|██████████████████████████████████████████████████| 3750/3750 [00:24<00:00, 150.68it/s]


0.13276297098224363
0.9927166666666836


loss 0.0185, accuracy 1.0: 100%|██████████████████████████████████████████████████| 3750/3750 [00:24<00:00, 150.52it/s]

0.1223796918756639
0.9933383333333508


In [90]:
accuracy(label, model(img))

<ipython-input-82-a8fa29b58036>:2: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  score = F.softmax(prediction.detach()).numpy().argmax(1) == label.numpy().argmax(1)


0.9375